In [1]:
!pip install -U transformers
!pip install -U accelerate
!pip install datasets
!pip install rouge_score

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 4.6 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

# Import

In [2]:
import numpy as np
import pandas as pd
import pickle
import math
import string

In [15]:
import json
import pandas as pd
import torch
import os
from PIL import Image
from torchvision.transforms import Compose, Resize, CenterCrop, ToTensor, Normalize
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
import torch.nn.functional as F
from torch import optim, nn
from tqdm import tqdm
from sklearn.metrics import classification_report
import numpy as np

# Data Read

In [51]:
from google.colab import drive
drive.mount('/content/drive', force_remount = True)

Mounted at /content/drive


In [4]:
with open("/content/drive/MyDrive/product_dict.pkl", 'rb') as file:
  df_product = pickle.load(file)

In [5]:
with open("/content/drive/MyDrive/all_detail_product.pkl", 'rb') as file:
  df_detail = pickle.load(file)

In [52]:
with open("/content/drive/MyDrive/df_pickle_model.pkl", 'rb') as file:
  df_full = pickle.load(file)


#EDA

In [6]:
def get_desc(lst):
  lst_str = ""
  for i in lst:
    lst_str = lst_str + str(i)
    lst_str = lst_str + str("\n")
  return lst_str

In [7]:
def fetch_price(text):
  # Remove the rupee symbol (₹) and commas (,)
  text_without_symbols = text.replace("₹", "").replace(",", "")

  # Convert the string to a float
  float_value = float(text_without_symbols)
  return float_value

In [113]:
def categorize_score(average_score):
  """
  Converts an average score between -1 and 1 to the closest category in [-1, -0.5, 0, 0.5, 1].

  Args:
      average_score: A float value between -1 and 1 representing the average score.

  Returns:
      The closest category in [-1, -0.5, 0, 0.5, 1] to the average score.
  """

  # Define the categories
  categories = [-1, -0.5, 0, 0.5, 1]
  # categories = [-0.25, -0.10, 0, 0.10, 0.25]

  # Find the closest category using absolute difference
  closest_index = min(range(len(categories)), key=lambda i: abs(categories[i] - average_score))
  # closest_score = (categories[closest_index])

  return closest_index

In [133]:
def categorize_rating(average_score):
  """
  Converts an average score between -1 and 1 to the closest category in [-1, -0.5, 0, 0.5, 1].

  Args:
      average_score: A float value between -1 and 1 representing the average score.

  Returns:
      The closest category in [-1, -0.5, 0, 0.5, 1] to the average score.
  """

  # Define the categories
  categories = [0, 1, 2, 3, 4, 5]
  # categories = [-0.25, -0.10, 0, 0.10, 0.25]

  # Find the closest category using absolute difference
  closest_index = min(range(len(categories)), key=lambda i: abs(categories[i] - average_score))
  # closest_score = (categories[closest_index])

  return closest_index

In [9]:
title = []
price = []
desc = []
score = []
image = []
id = []
for i in df_detail:
  id.append(i)
  title.append(df_detail[i][0])
  price.append(df_detail[i][1])
  # if df_detail[i][2] == " About this item ":
  desc.append(get_desc(df_detail[i][3]))
  score.append(categorize_score(df_detail[i][4]))
  image.append(df_product[i][2])
  # score.append(df_detail[i][4])
  # else:
    # desc.append(get_desc(df_detail[i][2]))
    # print(i)
    # score.append(categorize_score(df_detail[i][3]))

In [56]:
df_full.price.iloc[0]

''

In [70]:
df_ffull = df_full[~df_full.isin([''])]

In [71]:
df_ffull.dropna(inplace=True)
df_ffull = df_ffull.reset_index()

In [74]:
df_ffull.drop(["index"],axis = 1, inplace = True)
df_ffull

,asin,title,price,overall,sentiment,Image,description
0,B00004YKNF,Imation 3.5-Inch DS-HD IBM PC Formatted (Disco...,$20.85,4.766667,0.260190,<PIL.JpegImagePlugin.JpegImageFile image mode=...,"[Cost effective method of backing up, storing,..."
1,B00004Z5PY,Belkin Pro Series USB Device Cable 16 Feet,$10.79,4.601810,0.427029,<PIL.JpegImagePlugin.JpegImageFile image mode=...,[The Hi-Speed USB 2.0 cable helps you achieve ...
2,B0000511Z8,Tripp Lite 6ft KVM Switch USB Cable Kit for B0...,$9.86,4.500000,0.477150,<PIL.JpegImagePlugin.JpegImageFile image mode=...,[<P> Tripp Lite's 6-ft. USB KVM cable kit conn...
3,B00005KC6B,SONY DVP-NS300/B DVD Video Player (Black),$34.99,3.800000,0.642240,<PIL.JpegImagePlugin.JpegImageFile image mode=...,[A great deal for a great DVD player from Sony...
4,B00005MEN6,Olympus Camedia C-4040 4MP Digital Camera w/ 3...,$43.27,4.428571,0.742807,<PIL.JpegImagePlugin.JpegImageFile image mode=...,"[The Olympus C-4040 Zoom succeeds the C-3040, ..."
...,...,...,...,...,...,...,...
434,B01GVJO3V0,Skar Audio SK46 4&quot; x 6&quot; 120W 2-Way P...,$0.85,4.857143,0.760271,<PIL.JpegImagePlugin.JpegImageFile image mode=...,[The Skar Audio SK46 Performance Series 4-inch...
435,B01HD7EIVY,Sony AKA-FGP1&nbsp;Finger Grip for Action Cam,$1.98,4.666667,0.583893,<PIL.JpegImagePlugin.JpegImageFile image mode=...,[Get a grip with the easy to attach finger gri...
436,B01HE9R1HE,1byone 65 Miles Digital TV Antenna with Enhanc...,$27.70,4.214286,0.483954,<PIL.JpegImagePlugin.JpegImageFile image mode=...,[]
437,B01HGK8GIO,Mercurry 12V 2pin 25mm x25mm x10mm small mini ...,$7.98,3.571429,0.122443,<PIL.JpegImagePlugin.JpegImageFile image mode=...,[Package include: 2x 12V 2pin 25mm x25mm x10mm...


In [10]:
data = {
    "ID" : id,
    "Title" : title,
    "Price" : price,
    "Description" : desc,
    "Image" : image,
    "Score" : score
}

In [23]:
# Create DataFrame
df_sent = pd.DataFrame(data)

In [24]:
df_sent.dropna(inplace= True)

In [25]:
df_sent = df_sent[:19]

In [257]:
df_sent

,ID,Title,Price,Description,Image,Score
0,0,Apple iPhone 15 (128 GB) - Black,"₹73,100.00",DYNAMIC ISLAND COMES TO IPHONE 15 — Dynamic Is...,<PIL.JpegImagePlugin.JpegImageFile image mode=...,1
1,2,Apple iPhone 13 (128GB) - Starlight,"₹49,299.00",15 cm (6.1-inch) Super Retina XDR display\nCin...,<PIL.JpegImagePlugin.JpegImageFile image mode=...,4
2,4,Apple iPhone 15 (128 GB) - Blue,"₹73,100.00",DYNAMIC ISLAND COMES TO IPHONE 15 — Dynamic Is...,<PIL.JpegImagePlugin.JpegImageFile image mode=...,1
3,6,Apple iPhone 15 (128 GB) - Black,"₹73,100.00",DYNAMIC ISLAND COMES TO IPHONE 15 — Dynamic Is...,<PIL.JpegImagePlugin.JpegImageFile image mode=...,1
4,8,Apple iPhone 15 (256 GB) - Black,"₹82,600.00",DYNAMIC ISLAND COMES TO IPHONE 15 — Dynamic Is...,<PIL.JpegImagePlugin.JpegImageFile image mode=...,1
5,10,Apple iPhone 15 (128 GB) - Green,"₹73,100.00",DYNAMIC ISLAND COMES TO IPHONE 15 — Dynamic Is...,<PIL.JpegImagePlugin.JpegImageFile image mode=...,1
6,12,Apple iPhone 15 (128 GB) - Pink,"₹73,100.00",DYNAMIC ISLAND COMES TO IPHONE 15 — Dynamic Is...,<PIL.JpegImagePlugin.JpegImageFile image mode=...,1
7,14,Apple iPhone 15 (256 GB) - Blue,"₹82,600.00",DYNAMIC ISLAND COMES TO IPHONE 15 — Dynamic Is...,<PIL.JpegImagePlugin.JpegImageFile image mode=...,1
8,16,Apple iPhone 15 (256 GB) - Green,"₹82,600.00",DYNAMIC ISLAND COMES TO IPHONE 15 — Dynamic Is...,<PIL.JpegImagePlugin.JpegImageFile image mode=...,1
9,18,Apple iPhone 15 (256 GB) - Pink,"₹82,600.00",DYNAMIC ISLAND COMES TO IPHONE 15 — Dynamic Is...,<PIL.JpegImagePlugin.JpegImageFile image mode=...,1


In [114]:
df_sent = df_ffull.copy()

In [115]:
df_sent["Text"] = str("title: ") + str(df_sent["title"]) + str("\ndescription: ") + str(df_sent["description"]) + str("\nprice: ") + str(df_sent["price"])

In [78]:
df_sent.columns

Index(['asin', 'title', 'price', 'overall', 'sentiment', 'Image',
       'description'],
      dtype='object')

In [16]:
torch.cuda.empty_cache()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [17]:
torch.manual_seed(0)

In [18]:
from transformers import (
    VisionTextDualEncoderModel,
    VisionTextDualEncoderProcessor,
    AutoImageProcessor,
    AutoTokenizer,
)

In [19]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
image_processor = AutoImageProcessor.from_pretrained("google/vit-base-patch16-224")
processor = VisionTextDualEncoderProcessor(image_processor, tokenizer)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/160 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/69.7k [00:00<?, ?B/s]

#Average Sentiment Score predictior

In [37]:
modelSentiment = VisionTextDualEncoderModel.from_vision_text_pretrained(
    "google/vit-base-patch16-224", "bert-base-uncased"
)

Some weights of ViTModel were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized: ['vit.pooler.dense.bias', 'vit.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
The projection layer and logit scale weights `['visual_projection.weight', 'text_projection.weight', 'logit_scale']` are newly initialized. You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [21]:
class ClassificationModel(nn.Module):
  def __init__(self, model):
    super(ClassificationModel, self).__init__()
    self.model = model
    self.fc1 = nn.Linear(768 * 2, 128)
    self.fc2 = nn.Linear(128, 5)
    self.sig = nn.Sigmoid()
    self.softmax = nn.Softmax(dim=1)  # Softmax layer
    # self.fc3 = nn.Linear(64, 2)

  def forward(self, input_ids, pixel_values, attention_mask, token_type_ids):
    model_output = self.model(input_ids = input_ids, pixel_values = pixel_values, attention_mask = attention_mask, token_type_ids = token_type_ids)
    # x = torch.cat((model_output.text_embeds, model_output.image_embeds), -1)
    x = torch.cat((model_output.vision_model_output.pooler_output, model_output.text_model_output.pooler_output), -1)
    # x = model_output.vision_model_output.pooler_output * model_output.text_model_output.pooler_output
    x = self.fc1(x)
    # x = F.relu(self.fc2(x))
    x = self.fc2(x)
    x = self.softmax(x)  # Apply softmax after the final linear layer
    return x

  def tsne_features(self, input_ids, pixel_values, attention_mask, token_type_ids):
    model_output = self.model(input_ids = input_ids, pixel_values = pixel_values, attention_mask = attention_mask, token_type_ids = token_type_ids)
    x = torch.cat((model_output.vision_model_output.pooler_output, model_output.text_model_output.pooler_output), -1)
    # x = model_output.vision_model_output.pooler_output * model_output.text_model_output.pooler_output
    return x

In [ ]:
class RegressionModel(nn.Module):
  def __init__(self, model):
    super(RegressionModel, self).__init__()
    self.model = model
    self.fc1 = nn.Linear(768 * 2, 128)
    self.fc2 = nn.Linear(128, 1)  # Output layer for single regression value

  def forward(self, input_ids, pixel_values, attention_mask, token_type_ids):
    model_output = self.model(input_ids=input_ids, pixel_values=pixel_values, attention_mask=attention_mask, token_type_ids=token_type_ids)
    x = torch.cat((model_output.vision_model_output.pooler_output, model_output.text_model_output.pooler_output), -1)  # Concatenate text and image embeddings
    x = self.fc1(x)
    x = self.fc2(x)  # No activation in final layer for regression
    return x

  def tsne_features(self, input_ids, pixel_values, attention_mask, token_type_ids):
    model_output = self.model(input_ids=input_ids, pixel_values=pixel_values, attention_mask=attention_mask, token_type_ids=token_type_ids)
    x = torch.cat((model_output.vision_model_output.pooler_output, model_output.text_model_output.pooler_output), -1)
    return x

In [86]:
class ProductDataset(torch.utils.data.Dataset):
    def __init__(self, dataframe):
        self.frame = dataframe
        self.image_transform = Resize((256,256))

    def __len__(self):
        return len(self.frame)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        image = self.frame.loc[idx, 'Image']

        if image.mode != 'RGB':
            image = image.convert('RGB')

        image = self.image_transform(image)

        label = self.frame.loc[idx, 'sentiment']
        text = self.frame.loc[idx, 'Text']
        text_prompt = text
        inputs = processor(images = image, text = text_prompt, padding = 'max_length', max_length = 64, truncation = True, return_tensors="pt")
        inputs['input_ids'] = inputs['input_ids'].squeeze()
        inputs['token_type_ids'] = inputs['token_type_ids'].squeeze()
        inputs['attention_mask'] = inputs['attention_mask'].squeeze()
        inputs['pixel_values'] = inputs['pixel_values'].squeeze()
        # inputs['pixel_mask'] = inputs['pixel_mask'].squeeze()
        # sample = {"inputs": inputs, "label": torch.tensor(label, dtype = torch.int8)}
        return inputs, torch.tensor(label)


In [30]:
from sklearn.model_selection import train_test_split

In [95]:
df_train

,level_0,asin,title,price,overall,sentiment,Image,description,Text
0,0,B013MC8G62,Lowepro Slingshot Edge 150 AW Camera Case,$79.95,4.285714,0.453548,<PIL.JpegImagePlugin.JpegImageFile image mode=...,[the Slingshot Edge 150 AW is designed to keep...,title: 0 Imation 3.5-Inch DS-HD IBM PC Fo...
1,1,B00005KC6B,SONY DVP-NS300/B DVD Video Player (Black),$34.99,3.800000,0.642240,<PIL.JpegImagePlugin.JpegImageFile image mode=...,[A great deal for a great DVD player from Sony...,title: 0 Imation 3.5-Inch DS-HD IBM PC Fo...
2,2,B00009R6TA,Canon Deluxe Photo Backpack 200EG for Canon EO...,$38.95,4.470535,0.633966,<PIL.JpegImagePlugin.JpegImageFile image mode=...,[200EG Deluxe Camera Case. Brand: Canon USA. 6...,title: 0 Imation 3.5-Inch DS-HD IBM PC Fo...
3,3,B01CEFLFF2,Panasonic drops360&deg; LUXE Premium In-Ear St...,$2.04,4.023810,0.676262,<PIL.JpegImagePlugin.JpegImageFile image mode=...,"[Powerful, elegant and exceptionally powerful,...",title: 0 Imation 3.5-Inch DS-HD IBM PC Fo...
4,4,B001XUG3VQ,Belkin 6-Outlet Home/Office Surge Protector wi...,$10.43,3.583333,0.060667,<PIL.JpegImagePlugin.JpegImageFile image mode=...,[This Belkin Home/Office Series Surge Protecto...,title: 0 Imation 3.5-Inch DS-HD IBM PC Fo...
...,...,...,...,...,...,...,...,...,...
324,324,B0077E730M,Essential Accessories Kit For Canon PowerShot ...,$4.46,4.466667,0.518287,<PIL.JpegImagePlugin.JpegImageFile image mode=...,"[, This Kit Includes Some Of The Essential Acc...",title: 0 Imation 3.5-Inch DS-HD IBM PC Fo...
325,325,B00WUIKWGM,"Wasabi Power Canon LP-E6, LP-E6N Replacement (...",$1.98,4.459757,0.464277,<PIL.JpegImagePlugin.JpegImageFile image mode=...,[The Wasabi Power battery and charger kit incl...,title: 0 Imation 3.5-Inch DS-HD IBM PC Fo...
326,326,B016RBLUZE,Adafruit Metro Mini 328-5V 16MHz,$18.01,5.000000,0.467960,<PIL.JpegImagePlugin.JpegImageFile image mode=...,"[We love the ATmega328, and we use them a lot ...",title: 0 Imation 3.5-Inch DS-HD IBM PC Fo...
327,327,B01HD7EIVY,Sony AKA-FGP1&nbsp;Finger Grip for Action Cam,$1.98,4.666667,0.583893,<PIL.JpegImagePlugin.JpegImageFile image mode=...,[Get a grip with the easy to attach finger gri...,title: 0 Imation 3.5-Inch DS-HD IBM PC Fo...


In [116]:
df_sent["sentiment"] = df_sent["sentiment"].apply(categorize_score)

In [117]:
df_train, df_test = train_test_split(df_sent, test_size=0.25, random_state=42)

In [118]:
df_train["sentiment"].value_counts()

sentiment
3    288
2     25
4     16
Name: count, dtype: int64

In [119]:
df_train = df_train.reset_index()
df_train.drop(["index"], axis = 1, inplace = True)
df_test = df_test.reset_index()
df_test.drop(["index"], axis = 1, inplace = True)

In [120]:
train_dataset = ProductDataset(df_train)

In [121]:
test_dataset = ProductDataset(df_test)

In [122]:
BS = 4

In [123]:
trainloader = DataLoader(train_dataset, batch_size=BS,
                        shuffle=True, num_workers=0)

In [124]:
#Hyper Parameters
epochs = 1
lr = 0.01

In [125]:
modelSent = ClassificationModel(modelSentiment)
modelSent.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(modelSent.parameters(), lr=lr, weight_decay=1e-6)

In [126]:
#Training Loop
train_losses = []
train_acc = []
val_losses = []
val_acc = []
for i in range(epochs):
  print(f"******************************EPOCH - {i+1}****************************************")
  train_loss = 0
  val_loss = 0
  modelSent.train()
  correct = 0
  total = 0
  pred = []
  act = []

  for batch in tqdm(trainloader):
    inputs, labels = batch
    # print(set(labels))
    # labels = labels.long()
    # print(set(labels))
    inputs = inputs.to(device)
    labels = labels.to(device)
    # model_output = model(**inputs)
    # embeddings = torch.cat((model_output.text_embeds, model_output.image_embeds), -1)
    logits = modelSent(**inputs)
    loss = criterion(logits, labels)
    train_loss += loss.item()
    loss.backward()
    optimizer.step()
    # optimizer_clip.step()
    # optimizer_class.step()
    predicted = logits.argmax(1)
    correct += (predicted == labels).sum().item()
    total += labels.shape[0]
    pred.extend(predicted.cpu().detach().tolist())
    act.extend(labels.cpu().detach().tolist())
  print(set(pred))
  print(set(act))
  print(classification_report(act, pred))
  train_losses.append(train_loss)
  train_acc.append((correct/total))
  print("Train Accuracy: ", (correct/total))
  print("Train Loss: ", train_loss)

******************************EPOCH - 1****************************************


  0%|          | 0/83 [00:00<?, ?it/s]Unused or unrecognized kwargs: max_length, padding, truncation.
Unused or unrecognized kwargs: max_length, padding, truncation.
Unused or unrecognized kwargs: max_length, padding, truncation.
Unused or unrecognized kwargs: max_length, padding, truncation.
  1%|          | 1/83 [00:00<00:30,  2.69it/s]Unused or unrecognized kwargs: max_length, padding, truncation.
Unused or unrecognized kwargs: max_length, padding, truncation.
Unused or unrecognized kwargs: max_length, padding, truncation.
Unused or unrecognized kwargs: max_length, padding, truncation.
  2%|▏         | 2/83 [00:00<00:26,  3.08it/s]Unused or unrecognized kwargs: max_length, padding, truncation.
Unused or unrecognized kwargs: max_length, padding, truncation.
Unused or unrecognized kwargs: max_length, padding, truncation.
Unused or unrecognized kwargs: max_length, padding, truncation.
  4%|▎         | 3/83 [00:00<00:23,  3.41it/s]Unused or unrecognized kwargs: max_length, padding, trun

{3}
{2, 3, 4}
              precision    recall  f1-score   support

           2       0.00      0.00      0.00        25
           3       0.88      1.00      0.93       288
           4       0.00      0.00      0.00        16

    accuracy                           0.88       329
   macro avg       0.29      0.33      0.31       329
weighted avg       0.77      0.88      0.82       329

Train Accuracy:  0.8753799392097265
Train Loss:  85.92636674642563



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [127]:
testloader = DataLoader(test_dataset, batch_size=BS,
                        shuffle=True, num_workers=0)

In [128]:
test_losses = []
test_acc = []

modelSent.eval()
correct = 0
total = 0
pred = []
act = []
test_loss = 0
with torch.no_grad():
  for batch in testloader:
    inputs, labels = batch
    inputs = inputs.to(device)
    labels = labels.to(device)
    logits = modelSent(**inputs)
    loss = criterion(logits, labels)
    test_loss += loss.item()
    predicted = logits.argmax(1)
    correct += (predicted == labels).sum().item()
    total += labels.shape[0]
    pred.extend(predicted.cpu().detach().tolist())
    act.extend(labels.cpu().detach().tolist())
print(classification_report(act, pred))
print("test Accuracy: ", (correct/total))
print("test Loss: ", test_loss)
test_losses.append(test_loss)
test_acc.append((correct/total))

Unused or unrecognized kwargs: max_length, padding, truncation.
Unused or unrecognized kwargs: max_length, padding, truncation.
Unused or unrecognized kwargs: max_length, padding, truncation.
Unused or unrecognized kwargs: max_length, padding, truncation.
Unused or unrecognized kwargs: max_length, padding, truncation.
Unused or unrecognized kwargs: max_length, padding, truncation.
Unused or unrecognized kwargs: max_length, padding, truncation.
Unused or unrecognized kwargs: max_length, padding, truncation.
Unused or unrecognized kwargs: max_length, padding, truncation.
Unused or unrecognized kwargs: max_length, padding, truncation.
Unused or unrecognized kwargs: max_length, padding, truncation.
Unused or unrecognized kwargs: max_length, padding, truncation.
Unused or unrecognized kwargs: max_length, padding, truncation.
Unused or unrecognized kwargs: max_length, padding, truncation.
Unused or unrecognized kwargs: max_length, padding, truncation.
Unused or unrecognized kwargs: max_lengt

              precision    recall  f1-score   support

           2       0.00      0.00      0.00        25
           3       0.88      1.00      0.93       288
           4       0.00      0.00      0.00        16

    accuracy                           0.88       329
   macro avg       0.29      0.33      0.31       329
weighted avg       0.77      0.88      0.82       329

test Accuracy:  0.8753799392097265
test Loss:  85.35110169649124


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [153]:
with open("/content/drive/MyDrive/modelSentiment.pickle", 'wb') as file:
  pickle.dump(modelSent, file)

# Average Rating predictor

In [129]:
modelRating = VisionTextDualEncoderModel.from_vision_text_pretrained(
    "google/vit-base-patch16-224", "bert-base-uncased"
)

Some weights of ViTModel were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized: ['vit.pooler.dense.bias', 'vit.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
The projection layer and logit scale weights `['visual_projection.weight', 'text_projection.weight', 'logit_scale']` are newly initialized. You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [131]:
df_sent.columns

Index(['asin', 'title', 'price', 'overall', 'sentiment', 'Image',
       'description', 'Text'],
      dtype='object')

In [136]:
df_sent["overall"] = df_sent["overall"].apply(categorize_rating)

In [137]:
class ClassificationModel(nn.Module):
  def __init__(self, model):
    super(ClassificationModel, self).__init__()
    self.model = model
    self.fc1 = nn.Linear(768 * 2, 128)
    self.fc2 = nn.Linear(128, 6)
    self.sig = nn.Sigmoid()
    self.softmax = nn.Softmax(dim=1)  # Softmax layer
    # self.fc3 = nn.Linear(64, 2)

  def forward(self, input_ids, pixel_values, attention_mask, token_type_ids):
    model_output = self.model(input_ids = input_ids, pixel_values = pixel_values, attention_mask = attention_mask, token_type_ids = token_type_ids)
    # x = torch.cat((model_output.text_embeds, model_output.image_embeds), -1)
    x = torch.cat((model_output.vision_model_output.pooler_output, model_output.text_model_output.pooler_output), -1)
    # x = model_output.vision_model_output.pooler_output * model_output.text_model_output.pooler_output
    x = self.fc1(x)
    # x = F.relu(self.fc2(x))
    x = self.fc2(x)
    x = self.softmax(x)  # Apply softmax after the final linear layer
    return x

  def tsne_features(self, input_ids, pixel_values, attention_mask, token_type_ids):
    model_output = self.model(input_ids = input_ids, pixel_values = pixel_values, attention_mask = attention_mask, token_type_ids = token_type_ids)
    x = torch.cat((model_output.vision_model_output.pooler_output, model_output.text_model_output.pooler_output), -1)
    # x = model_output.vision_model_output.pooler_output * model_output.text_model_output.pooler_output
    return x

In [ ]:
class RegressionModel(nn.Module):
  def __init__(self, model):
    super(RegressionModel, self).__init__()
    self.model = model
    self.fc1 = nn.Linear(768 * 2, 128)
    self.fc2 = nn.Linear(128, 1)  # Output layer for single regression value

  def forward(self, input_ids, pixel_values, attention_mask, token_type_ids):
    model_output = self.model(input_ids=input_ids, pixel_values=pixel_values, attention_mask=attention_mask, token_type_ids=token_type_ids)
    x = torch.cat((model_output.vision_model_output.pooler_output, model_output.text_model_output.pooler_output), -1)  # Concatenate text and image embeddings
    x = self.fc1(x)
    x = self.fc2(x)  # No activation in final layer for regression
    return x

  def tsne_features(self, input_ids, pixel_values, attention_mask, token_type_ids):
    model_output = self.model(input_ids=input_ids, pixel_values=pixel_values, attention_mask=attention_mask, token_type_ids=token_type_ids)
    x = torch.cat((model_output.vision_model_output.pooler_output, model_output.text_model_output.pooler_output), -1)
    return x

In [138]:
class ProductDataset(torch.utils.data.Dataset):
    def __init__(self, dataframe):
        self.frame = dataframe
        self.image_transform = Resize((256,256))

    def __len__(self):
        return len(self.frame)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        image = self.frame.loc[idx, 'Image']

        if image.mode != 'RGB':
            image = image.convert('RGB')

        image = self.image_transform(image)

        label = self.frame.loc[idx, 'overall']
        text = self.frame.loc[idx, 'Text']
        text_prompt = text
        inputs = processor(images = image, text = text_prompt, padding = 'max_length', max_length = 64, truncation = True, return_tensors="pt")
        inputs['input_ids'] = inputs['input_ids'].squeeze()
        inputs['token_type_ids'] = inputs['token_type_ids'].squeeze()
        inputs['attention_mask'] = inputs['attention_mask'].squeeze()
        inputs['pixel_values'] = inputs['pixel_values'].squeeze()
        # inputs['pixel_mask'] = inputs['pixel_mask'].squeeze()
        # sample = {"inputs": inputs, "label": torch.tensor(label, dtype = torch.int8)}
        return inputs, torch.tensor(label)


In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
df_train

,level_0,asin,title,price,overall,sentiment,Image,description,Text
0,0,B013MC8G62,Lowepro Slingshot Edge 150 AW Camera Case,$79.95,4.285714,0.453548,<PIL.JpegImagePlugin.JpegImageFile image mode=...,[the Slingshot Edge 150 AW is designed to keep...,title: 0 Imation 3.5-Inch DS-HD IBM PC Fo...
1,1,B00005KC6B,SONY DVP-NS300/B DVD Video Player (Black),$34.99,3.800000,0.642240,<PIL.JpegImagePlugin.JpegImageFile image mode=...,[A great deal for a great DVD player from Sony...,title: 0 Imation 3.5-Inch DS-HD IBM PC Fo...
2,2,B00009R6TA,Canon Deluxe Photo Backpack 200EG for Canon EO...,$38.95,4.470535,0.633966,<PIL.JpegImagePlugin.JpegImageFile image mode=...,[200EG Deluxe Camera Case. Brand: Canon USA. 6...,title: 0 Imation 3.5-Inch DS-HD IBM PC Fo...
3,3,B01CEFLFF2,Panasonic drops360&deg; LUXE Premium In-Ear St...,$2.04,4.023810,0.676262,<PIL.JpegImagePlugin.JpegImageFile image mode=...,"[Powerful, elegant and exceptionally powerful,...",title: 0 Imation 3.5-Inch DS-HD IBM PC Fo...
4,4,B001XUG3VQ,Belkin 6-Outlet Home/Office Surge Protector wi...,$10.43,3.583333,0.060667,<PIL.JpegImagePlugin.JpegImageFile image mode=...,[This Belkin Home/Office Series Surge Protecto...,title: 0 Imation 3.5-Inch DS-HD IBM PC Fo...
...,...,...,...,...,...,...,...,...,...
324,324,B0077E730M,Essential Accessories Kit For Canon PowerShot ...,$4.46,4.466667,0.518287,<PIL.JpegImagePlugin.JpegImageFile image mode=...,"[, This Kit Includes Some Of The Essential Acc...",title: 0 Imation 3.5-Inch DS-HD IBM PC Fo...
325,325,B00WUIKWGM,"Wasabi Power Canon LP-E6, LP-E6N Replacement (...",$1.98,4.459757,0.464277,<PIL.JpegImagePlugin.JpegImageFile image mode=...,[The Wasabi Power battery and charger kit incl...,title: 0 Imation 3.5-Inch DS-HD IBM PC Fo...
326,326,B016RBLUZE,Adafruit Metro Mini 328-5V 16MHz,$18.01,5.000000,0.467960,<PIL.JpegImagePlugin.JpegImageFile image mode=...,"[We love the ATmega328, and we use them a lot ...",title: 0 Imation 3.5-Inch DS-HD IBM PC Fo...
327,327,B01HD7EIVY,Sony AKA-FGP1&nbsp;Finger Grip for Action Cam,$1.98,4.666667,0.583893,<PIL.JpegImagePlugin.JpegImageFile image mode=...,[Get a grip with the easy to attach finger gri...,title: 0 Imation 3.5-Inch DS-HD IBM PC Fo...


In [139]:
df_train, df_test = train_test_split(df_sent, test_size=0.25, random_state=42)

In [140]:
df_train["overall"].value_counts()

overall
4    191
5    115
3     21
2      2
Name: count, dtype: int64

In [141]:
df_train = df_train.reset_index()
df_train.drop(["index"], axis = 1, inplace = True)
df_test = df_test.reset_index()
df_test.drop(["index"], axis = 1, inplace = True)

In [142]:
train_dataset = ProductDataset(df_train)

In [143]:
test_dataset = ProductDataset(df_test)

In [144]:
BS = 4

In [145]:
trainloader = DataLoader(train_dataset, batch_size=BS,
                        shuffle=True, num_workers=0)

In [146]:
#Hyper Parameters
epochs = 1
lr = 0.01

In [149]:
modelRat = ClassificationModel(modelRating)
modelRat.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(modelRat.parameters(), lr=lr, weight_decay=1e-6)

In [150]:
#Training Loop
train_losses = []
train_acc = []
val_losses = []
val_acc = []
for i in range(epochs):
  print(f"******************************EPOCH - {i+1}****************************************")
  train_loss = 0
  val_loss = 0
  modelRat.train()
  correct = 0
  total = 0
  pred = []
  act = []

  for batch in tqdm(trainloader):
    inputs, labels = batch
    # print(set(labels))
    # labels = labels.long()
    # print(set(labels))
    inputs = inputs.to(device)
    labels = labels.to(device)
    # model_output = model(**inputs)
    # embeddings = torch.cat((model_output.text_embeds, model_output.image_embeds), -1)
    logits = modelRat(**inputs)
    loss = criterion(logits, labels)
    train_loss += loss.item()
    loss.backward()
    optimizer.step()
    # optimizer_clip.step()
    # optimizer_class.step()
    predicted = logits.argmax(1)
    correct += (predicted == labels).sum().item()
    total += labels.shape[0]
    pred.extend(predicted.cpu().detach().tolist())
    act.extend(labels.cpu().detach().tolist())
  print(set(pred))
  print(set(act))
  print(classification_report(act, pred))
  train_losses.append(train_loss)
  train_acc.append((correct/total))
  print("Train Accuracy: ", (correct/total))
  print("Train Loss: ", train_loss)

******************************EPOCH - 1****************************************


  0%|          | 0/83 [00:00<?, ?it/s]Unused or unrecognized kwargs: max_length, padding, truncation.
Unused or unrecognized kwargs: max_length, padding, truncation.
Unused or unrecognized kwargs: max_length, padding, truncation.
Unused or unrecognized kwargs: max_length, padding, truncation.
  1%|          | 1/83 [00:00<00:31,  2.61it/s]Unused or unrecognized kwargs: max_length, padding, truncation.
Unused or unrecognized kwargs: max_length, padding, truncation.
Unused or unrecognized kwargs: max_length, padding, truncation.
Unused or unrecognized kwargs: max_length, padding, truncation.
  2%|▏         | 2/83 [00:00<00:27,  2.94it/s]Unused or unrecognized kwargs: max_length, padding, truncation.
Unused or unrecognized kwargs: max_length, padding, truncation.
Unused or unrecognized kwargs: max_length, padding, truncation.
Unused or unrecognized kwargs: max_length, padding, truncation.
  4%|▎         | 3/83 [00:01<00:26,  3.07it/s]Unused or unrecognized kwargs: max_length, padding, trun

{0, 1, 4, 5}
{2, 3, 4, 5}
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       0.00      0.00      0.00         0
           2       0.00      0.00      0.00         2
           3       0.00      0.00      0.00        21
           4       0.33      0.02      0.03       191
           5       0.34      0.94      0.50       115

    accuracy                           0.34       329
   macro avg       0.11      0.16      0.09       329
weighted avg       0.31      0.34      0.19       329

Train Accuracy:  0.3373860182370821
Train Loss:  141.55997371673584



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification

In [151]:
testloader = DataLoader(test_dataset, batch_size=BS,
                        shuffle=True, num_workers=0)

In [152]:
test_losses = []
test_acc = []

modelRat.eval()
correct = 0
total = 0
pred = []
act = []
test_loss = 0
with torch.no_grad():
  for batch in testloader:
    inputs, labels = batch
    inputs = inputs.to(device)
    labels = labels.to(device)
    logits = modelRat(**inputs)
    loss = criterion(logits, labels)
    test_loss += loss.item()
    predicted = logits.argmax(1)
    correct += (predicted == labels).sum().item()
    total += labels.shape[0]
    pred.extend(predicted.cpu().detach().tolist())
    act.extend(labels.cpu().detach().tolist())
print(classification_report(act, pred))
print("test Accuracy: ", (correct/total))
print("test Loss: ", test_loss)
test_losses.append(test_loss)
test_acc.append((correct/total))

Unused or unrecognized kwargs: max_length, padding, truncation.
Unused or unrecognized kwargs: max_length, padding, truncation.
Unused or unrecognized kwargs: max_length, padding, truncation.
Unused or unrecognized kwargs: max_length, padding, truncation.
Unused or unrecognized kwargs: max_length, padding, truncation.
Unused or unrecognized kwargs: max_length, padding, truncation.
Unused or unrecognized kwargs: max_length, padding, truncation.
Unused or unrecognized kwargs: max_length, padding, truncation.
Unused or unrecognized kwargs: max_length, padding, truncation.
Unused or unrecognized kwargs: max_length, padding, truncation.
Unused or unrecognized kwargs: max_length, padding, truncation.
Unused or unrecognized kwargs: max_length, padding, truncation.
Unused or unrecognized kwargs: max_length, padding, truncation.
Unused or unrecognized kwargs: max_length, padding, truncation.
Unused or unrecognized kwargs: max_length, padding, truncation.
Unused or unrecognized kwargs: max_lengt

              precision    recall  f1-score   support

           3       0.00      0.00      0.00         6
           4       0.00      0.00      0.00        60
           5       0.40      1.00      0.57        44

    accuracy                           0.40       110
   macro avg       0.13      0.33      0.19       110
weighted avg       0.16      0.40      0.23       110

test Accuracy:  0.4
test Loss:  45.720574378967285


In [155]:
with open("/content/drive/MyDrive/modelRat.pickle", 'wb') as file:
  pickle.dump(modelRat, file)